In [1]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/attentionprediction'''

Mounted at /content/drive


In [2]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [4]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_IVSN','h_EccNet','h_NoTarget','h_Layer']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

conn = sqlite3.connect(ROOT+"/VS_Ablated/ec2_db/turing_vs_ablated.db")
cursor = conn.cursor()
cursor.execute("select * from newdb;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'eccnet':1,'notarget':1,'ivsn':1,'gbvs':1,'deepgaze':1,'swap':1,'catch':1,'chatgpt':1,'layer':1}
modelname_dict= {'ivsn':'IVSN','gbvs':'GBVS','deepgaze':'DeepGazeIII','chatgpt':"ChatGPT",'swap':'SwappedBackground','catch':'CatchTrial','eccnet':'EccNet','notarget':'NoTarget','layer':'Layer'}
count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []

dummy_marks_overall = []
confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue

        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 40
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST' and \
                dat['trialdata']['object response'] == dat['trialdata']['correct object']:
                dummy_mark += 1
        if dummy_mark < threshold:

            print("====="*20)
            print("[Dummy Test Failed]", dummy_mark)
            continue
        dummy_marks_overall.append(dummy_mark)
        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))



        participant_count += 1

        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                count += 1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trial_data = trial_data['trialData']
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    model = 'catch'
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    model = 'swap'
                else:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
#                         print(trial_data['imageID'])

                        model = trial_data['trialData']['imageID'].split('/')[-2].split('_')[-1] if 'Ablated' not in trial_data['trialData']['imageID'] else trial_data['trialData']['imageID'].split('/')[-2].split('_')[-2]
#                         print(model)
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
#                     marks.append(1 if trial_data['correct response'] == trial_data['choose response'] else 0)
#                     marks_conf.append(deepcopy(confmatrices['h_machine']))
#                 else:

#                 else:
#                     print('catch trial')



                # clear inconsistency
                trial_data['trialData']['response_gender'] = trial_data['trialData']['gender response']
                del trial_data['trialData']['gender response']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['object response'].lower()
                del trial_data['trialData']['object response']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['correct object'].lower()
                del trial_data['trialData']['correct object']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
                trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_VS_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)

        print(confmatrices)
        print('catch',catch_trial_mat)
        print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-04-21 06:20:54
[Time Used] 12.2 mins
[Questionnaire] {'country': 'United States', 'age': '46-50', 'engagement': '10', 'difficulty': '3', 'mode': 'live', 'gender': 'female', 'education': 'bachelor', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 33/40
{'h_machine': array([[11.,  9.],
       [10., 10.]]), 'h_IVSN': array([[3., 2.]]), 'h_EccNet': array([[3., 2.]]), 'h_NoTarget': array([[3., 2.]]), 'h_Layer': array([[1., 4.]])}
catch [[0. 0.]]
swap [[0. 0.]]
[Finished Date] 2025-04-21 06:17:49
[Time Used] 7.5 mins
[Questionnaire] {'country': 'United States', 'age': '46-50', 'engagement': '7', 'difficulty': '9', 'mode': 'live', 'gender': 'male', 'education': 'master', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 28/40
{'h_machine': array([[ 9., 11.],
       [ 8., 12.]]), 'h_IVSN': array([[2., 3.]]), 'h_EccNet': array([[1., 4.]]), 'h_NoTarget': array([[2., 3.]]), 'h_Layer': array([[

In [ ]:
trial_data['trialData']['imageID']

'https://xiaoturing.s3.us-east-1.amazonaws.com/Ablated_VS/Ablated_EccNet_GIF/bg_img106_scanpath_Eccnet.gif'

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/attention_prediction_task/Plot/Data_all/complete_humanjudge_VS_Ablated.json", 'w') as f:
    json.dump(trials_dict,f)
with open("/home/liuxiao/TuringGithub/XiaoData/complete_humanjudge_VS_Ablated.json", 'w') as f:
    json.dump(trials_dict,f)

In [ ]:
confmat_overall

{'h_machine': array([[547., 413.],
        [421., 539.]]),
 'h_IVSN': array([[116., 124.]]),
 'h_EccNet': array([[115., 125.]]),
 'h_NoTarget': array([[105., 135.]]),
 'h_Layer': array([[ 85., 155.]])}

Bad pipe message: %s [b'7\x1a\xa7K\xd1S\x98c\x98\x87\xa1\xcb\xa9\xb0,\xf1\xbf9\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89']
Bad pipe message: %s [b'\xaflu#\xfa\x12{\x8f\x98\xb0\x0fu\x96@k\xea\xa6T\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.

In [5]:
len(trials_dict)/len(dummy_marks_overall)

40.0

In [7]:
np.mean(dummy_marks_overall)/40

np.float64(0.7489583333333333)

In [ ]:
catch_trial_mat_overall

array([[0., 0.]])

In [ ]:
534/(466.+ 534.)

0.534